This notebook provides an algorithm for finding all monogenizations of a quartic order defined by a polynomial.
A ring is monogenic if it is generated by a single element as a $\mathbb{Z}$ algebra.
If $R$ is a monogenic ring, with $\mathbb{Z}$ algebra generator $\xi$, then we say $\xi$ is a monogenizer of $R$.
Note that if $\xi$ is a monogenizer of a ring $R$ then $\pm \xi + c$ will also be a monogenizer for any $c \in \mathbb{Z}$, in this case we say that $\xi$ and $\pm \xi +c$ are $\mathbb{Z}$ equivalent.
A $\mathbb{Z}$ equivalence class is called a monogenization.


Let $f \in \mathbb{Z}[t]$ be irreducible and monic, then the ring $R=\mathbb{Z}[t]/(f(t))$ is a domain and is generated by $t$ as a $\mathbb{Z}$ algebra.
If $\xi$ is any root of $f$ then $R$ will be isomorphic to $\mathbb{Z}[\xi]$. 
The code here is an implementation of the algorithm given by István Gaál, Attila Pethő, and Michael Pohst (see https://www.sciencedirect.com/science/article/pii/S0022314X96900359).
The output will be a triple $(x, y, z) \in \mathbb{Z}^3$ which gives the common coordinates for every element of a monogenization of $\mathbb{Z}[\xi]$ in the basis $\{1, \xi, \xi^2, \xi^3\}$.





Code was written for SageMath 10.5

This notebook provides an algorithm for finding all monogenizations of a quartic order defined by a polynomial.  

A ring is \textbf{monogenic} if it is generated by a single element as a $\mathbb{Z}$-algebra. If $R$ is a monogenic ring with $\mathbb{Z}$-algebra generator $\xi$, then we say that $\xi$ is a \textbf{monogenizer} of $R$.  

Note that if $\xi$ is a monogenizer of a ring $R$, then $\pm \xi + c$ will also be a monogenizer for any $c \in \mathbb{Z}$. In this case, we say that $\xi$ and $\pm \xi + c$ are \textbf{$\mathbb{Z}$-equivalent}. A $\mathbb{Z}$-equivalence class is called a \textbf{monogenization}.  

Let $f \in \mathbb{Z}[t]$ be an irreducible and monic polynomial. Then the ring $R = \mathbb{Z}[t] / (f(t))$ is a domain and is generated by $t$ as a $\mathbb{Z}$-algebra. If $\xi$ is any root of $f$, then $R$ is isomorphic to $\mathbb{Z}[\xi]$.  

The code in this notebook implements the algorithm given by István Gaál, Attila Pethő, and Michael Pohst (see \href{https://www.sciencedirect.com/science/article/pii/S0022314X96900359}{this paper}). The output is a triple $(x, y, z) \in \mathbb{Z}^3$ that provides the common coordinates for every element of a monogenization of $\mathbb{Z}[\xi]$ in the basis $\{1, \xi, \xi^2, \xi^3\}$.  

This code was written for \textbf{SageMath 10.5}.  




In [1]:
#Run this cell first

# import random
import itertools
from sage.quadratic_forms.qfsolve import qfsolve


A= ZZ['t']
t=A.gen()
# It is needed to use some SageMath functions, particularly the is_irreducible () function.
R.<T, S, X , Y, Z> = PolynomialRing(QQ)
#This ring is needed when solving thue equations
# P.<X,Y,Z> =QQ[]

#For now, this has a dictionary with every irreducible poly in the selected range. Later, this will hopefully involve some spl, ideas
# The key C returns an array of the coefficients checked so far.


def solvethue(f,val_list):
    # This calculation takes a polynomial and a list of values, and finds all solutions to the thue equation F=val, for val in val_list
    #we are using the GRH flag to speed up the calculation, and we then check the results
    out=[]
    F=gp.thueinit(f,1)
    for val in val_list:
        out=out+gp.thue(F,val).sage()
    return out
#This function will find all integral solutions to a thue equation, it calls the thue solver in Pair/gp

# def solvethue(f,n):
#     assert f.is_homogeneous()
#     return gp.thue( gp.thueinit(f), n).sage()

def Qform_solve(M):
# The input here is a Gram matrix of a trinary quadratic form
    sol= qfsolve(M)
    #qfsolve will return a solution to the quadratic form or an integer corresponding to the place where the form is unsolvable
    if isinstance(sol, (int, Integer)):
        return(False)
    else:
        return(sol)

####################################################################################################################################
#this class if for a quartic thue equation, it is used in the main class poly.
class quartic_thue:
    def __init__(self):
        self.equation = None
        self.fx = None
        self.fy = None
        self.fz= None
        self.k_list = []
        
    
    def reset(self):
        self.__init__()



#################################################################################################################################3
class poly:
    def __init__(self, coeff ):
        self.coeff =coeff 
        # an array that records the coefficients of the poly,
        # we choose to have five entries even though the first is always 1, 
        #to make recalling coefficients index match mathematical labeling
        self.cubic_res= self.cubic_resolvent()
        #a trinary quadratic form coming from the index form equation
        self.Q1= self.index_equations(1)
        # a trinary quadratic from coming from the index form equation
        self.Q2=self.index_equations(2)
        #the coord
        self.cubic_coords= set()
        self.solu_coords={}
        #a dictionary that records an array of the coordinates attached to each cubic solution
        # This dictionary has keys, the cubic solutions, and the values for each key are a list of the coordinates defining that cubic solution
        self.quar_number=0
        self.cubic_number=0
        self.QT=quartic_thue()
        #This holds a binary quartic form and information about how it relates to the solution to the index form
        #it is reset after each use to reduce the memory need for the whole calculation
        self.signature=None

        
        
    def reset(self,new_coeff):
         self.__init__(new_coeff)
        
 
    def cubic_resolvent(self):
            return( [1,-self.coeff[2], self.coeff[1]*self.coeff[3] - 4*self.coeff[4], 
                     4*self.coeff[2]*self.coeff[4]-self.coeff[3]**2-self.coeff[1]**2*self.coeff[4]])
            
    
    def index_equations(self,i):
        A=self.coeff
        if i ==1:
            Q_1=X**2-A[1]*X*Y + A[2]*Y**2+X*Z*(A[1]**2-2*A[2])+Y*Z*(A[3]-A[1]*A[2])+Z**2*(-A[1]*A[3]+A[2]**2 +A[4])
            return(Q_1)
        if i ==2:
            Q_2 = Y**2 - X*Z - A[1]*Y*Z+A[2]*Z**2
            return(Q_2)

    
    def make(self, coeff_list):
        #converts a list of coefficients to a univariable polynomial for operation in Sage
        L=len(coeff_list)
        f= sum((coeff_list[i])* t**(L-1-i) for i in range(L))
        return(f)
    
    def make_form(self, coeff_list):
        #converts a list of coeff to a binary form
        L=len(coeff_list)
        F= sum(coeff_list[i]* (S**i)*(T**(L-1-i)) for i in range(L))
        return(F)
    
    def typeI_cubic(self):
        # This should find the solution to the cubic thue
            c_list= self.cubic_res
            L=len(c_list)
            f= self.make(self.cubic_res)
            Solu=solvethue(f,[1])
            # Since f is an odd functio,n we only need to check the positive solutions
            self.cubic_numberr=len(Solu)
            for I in Solu:
                t_1=I[0]
                t_2=I[1]
                self.solu_coords[(t_1,t_2)]=False
                point = (t_1+c_list[1]*t_2, t_2)
                self.cubic_coords.add(point)
                # The value is false, let us know that we have not tested for quar monogen over this cubic monogen yet.
    
    def find_quar_thue(self,sol_cub):
        if sol_cub==(1,0) or sol_cub== (-1,0):
            f_x=T**2 + self.coeff[1]*T*S+ self.coeff[2]*S**2
            f_y = T*S+ self.coeff[1]*S**2
            f_z = S**2
            self.QT.equation= self.make_form(self.coeff)
            self.QT.fx=f_x
            self.QT.fy=f_y
            self.QT.fz=f_z
            self.QT.k_list=[1]
            return(True)
        else:
            u=sol_cub[0]
            v=sol_cub[1]
            
            Q_1=self.Q1
            Q_2 = self.Q2
            # These are the quadratic forms, from the index system of equations
            d, l,k = xgcd(u,v)
            Q_u=l*Q_1+k*Q_2
            Q_v = v*Q_1-u*Q_2
            #We are finding the coefficient of Q_v
            # This section could be improved
            a=Q_v.subs(X=1,Y=0, Z=0)
            b=Q_v.subs(X=1, Y=1, Z=0) - Q_v.subs(X=1, Y=0, Z=0) -Q_v.subs(X=0, Y=1, Z=0) 
            c= Q_v.subs(X=1, Y=0,Z=1) - Q_v.subs(X=1, Y=0, Z=0)- Q_v.subs(X=0, Y=0, Z=1)
            d= Q_v.subs(X=0, Y=1,Z=0)
            e=Q_v.subs(X=0, Y=1,Z=1) -Q_v.subs(X=0, Y=1, Z=0) - Q_v.subs(X=0, Y=0, Z=1) 
            f=Q_v.subs(X=0, Y=0,Z=1)
            M_v=Matrix(QQ, [[a, b/2, c/2], [b/2, d, e/2], [c/2, e/2, f]])
            qfsol=Qform_solve(M_v)
            
            if qfsol== False:
                self.QT.reset()
                return(False)
            # This is one of the fail points for no quartic monogenizers existing for a given cubic monogenizer

            else:
                x_0=qfsol[0] ; y_0= qfsol[1] ; z_0= qfsol[2]
                # The next polynomial parameterizes the solution to Q_v=0
                #note only one of y_0 or z_0 can be zero else our cubic solution is (1,0)
                if z_0!=0:
                    L_1=2*a*x_0 + b*y_0 +c*z_0 ; L_2 = b*x_0 + 2*d*y_0 + e*z_0
                    f_x= (-a*x_0 + L_1)*T**2 + (-b*x_0 + L_2)*T*S + (-d*x_0)*S**2
                    f_y= (-a*y_0)*T**2+ (-b*y_0 + L_1)*T*S+ (-d*y_0+L_2)*S**2
                    f_z = (-a*z_0)*T**2 + (-b*z_0)*T*S + -d*z_0*S**2
                else:
                    L_1=2*a*x_0 + b*y_0 + c*z_0 ; L_2 = c*x_0 + e*y_0 + 2*f*z_0
                    f_x= (-a*x_0 +L_1)*T**2 + (-c*x_0 + L_2)*T*S + (-f*x_0)*S**2
                    f_y = (-a*y_0)*T**2 + (-c*y_0)*T*S + (-f*y_0)*S**2
                    f_z = (-a*z_0)*T**2 + (-c*z_0+ L_1)*T*S + (-f*z_0 + L_2)*S**2
                #now we need to find the value of the multiple k
                coeff_x = [f_x.subs(T=1, S=0), f_x.subs(T=1, S=1)- f_x.subs(T=1, S=0) -f_x.subs(T=0, S=1), f_x.subs(T=0, S=1)]
                coeff_y = [f_y.subs(T=1, S=0), f_y.subs(T=1, S=1)- f_y.subs(T=1, S=0) -f_y.subs(T=0, S=1), f_y.subs(T=0, S=1)]
                coeff_z = [f_z.subs(T=1, S=0), f_z.subs(T=1, S=1)- f_z.subs(T=1, S=0) -f_z.subs(T=0, S=1), f_z.subs(T=0, S=1)]

                coeff_mat=matrix([coeff_x, coeff_y, coeff_z])
                mult=coeff_mat.det()/(gcd(coeff_x+coeff_y+coeff_z))
                self.QT.k_list=divisors(mult)
                

                #here, we are recording the information that is needed to find the coordinates
                self.QT.equation= Q_2.subs(X=f_x, Y=f_y, Z= f_z)
                self.QT.fx=f_x
                self.QT.fy=f_y
                self.QT.fz=f_z

                return(True)

            
            
    def coord_check(self, point, sol_cub):
        #         bool1=((P.Q1.subs(X=point[0],Y=point[1],Z=point[2]),P.Q2.subs(X=point[0],Y=point[1],Z=point[2])) == sol_cub) 
        # bool2=((-1*P.Q1.subs(X=point[0],Y=point[1],Z=point[2]),-1*P.Q2.subs(X=point[0],Y=point[1],Z=point[2])) == sol_cub)
        bool1=((self.Q1.subs(X=point[0],Y=point[1],Z=point[2]),self.Q2.subs(X=point[0],Y=point[1],Z=point[2])) == sol_cub) 
        bool2=((-1*self.Q1.subs(X=point[0],Y=point[1],Z=point[2]),-1*self.Q2.subs(X=point[0],Y=point[1],Z=point[2])) == sol_cub)
        return(bool1 or bool2)
    
    def typeI_quar(self):
        # This function takes a solution to the cubic and finds all coordinates that give a quartic monogenizer over that cubic
        #the output is stored as a value in the dictionary self.coords
        # we store the value of True if no solution is found, this indicates that the test was run
        for sol_cub in self.solu_coords.keys():
            if self.solu_coords[sol_cub]==False:
                val=self.find_quar_thue(sol_cub)
                if val== True:
                    f=self.QT.equation
                    if sol_cub[1]==0:
                        quar_solutions=solvethue(f.subs(S=1),[-1,1])
                    elif sol_cub[1]!= 0:
                        val_list=self.QT.k_list +[-1* i for i in self.QT.k_list]
                        quar_solutions=solvethue(f.subs(S=1),val_list)
                        
                    if quar_solutions==[]:
                        self.solu_coords[sol_cub]= 0
                        #the true here indicates that the val has been tested and that no solutions were found
                    else:
                        coord_list=[]
                        for I in quar_solutions:
                            t_1=I[0]
                            s_1=I[1]
                            point=(self.QT.fx.subs(T=t_1, S=s_1), self.QT.fy.subs(T=t_1, S=s_1), self.QT.fz.subs(T=t_1, S=s_1))
                            if self.coord_check(point,sol_cub):
                                coord_list.append(point)
                        coord_set= set(coord_list)
                        self.solu_coords[sol_cub]=coord_set
                        self.quar_number= self.quar_number + len(coord_set)
                elif val==False:
                    self.solu_coords[sol_cub]= 0
                    #the true here indicates that the val has been tested and that no solution were found

    
    def find_signature(self):
        out=[]
        for I in self.solu_coords.keys():
            print(I)
            if self.solu_coords[I]==0:
                out.append(0)
            else:
                out.append(len(self.solu_coords[I]))
        print(out)
        self.signature=out
        
    def monogen_test(self):
        #calling this method will find all the mongenizations of the ring defined by the input polynomial
        #we will return the signature which 
        self.typeI_cubic()
        self.typeI_quar()
        self.find_signature()
        return(self.signature)



The next cell contains several usefull function for generating the datasets of quartic orders.

In [ ]:


########################################################################################################################################

def find_monogens(num_list):
    if len(num_list)==5:
        print('test')
        P=poly(num_list)
        f_p=P.make(P.coeff)
        if f_p.is_irreducible(): 
            P.monogen_test()
            print( P.solu_coords)
            # print(P.signature)
        else:
            print(f'the polynomial defined by {num_list} is reducible')
    else:
        print('the polynomial must be degree 4')

##############################################################################################################################
def type_II_test(nums):
    #take a list of checks and check a necessary condition for the associated order to be type II
    # The function returns false if the order is not type II and true if it might be type II
    P=poly(nums)
    P.monogen_test()
    M=list(P.solu_coords.values())
    for s in M:
        if isinstance(s, set):  
            for e in s:
                if (e[2]==0) and (e != (1,0,0)):
                    return(True)
    return(False)
###########################################################################################################################################
def gal_group(nums):
    # Finds the Galois group of the polynomial with coefficients coming from the array nums
    P=poly(nums)
    f_p = P.make(P.coeff)
    r_p = P.make(P.cubic_res)
    if r_p.is_irreducible():
        if r_p.discriminant().is_square():
            return('A_4')
        else:
            return('S_4')
    else:
        if len(r_p.factor())== 3:
            return('V_4')
        else:
            K.<a> = r_p.splitting_field()
            R.<x> = K[]
            rational_roots=r_p.roots(QQ, multiplicities = False)
            root= rational_roots[0]
            g= (x**2 - root*x + P.coeff[4])*(x**2 + P.coeff[1]*x+(P.coeff[2]-root))
            number_of_roots= sum(point[1] for point in  g.roots(K, multiplicities=True))
            check =(number_of_roots ==4)
            if check:
                return('C_4')
            else:
                return('D_4')


#############################################################################################################################################
def ring_int_test(nums):
    #test if the ring defined by the polynomial with coefficients from nums is a maximal order
    p=poly(nums)
    f = p.make(p.coeff)
    K.<a> = NumberField(f)
    Za= K.order([1, a, a^2, a^3])
    OK = K.maximal_order()
    return(Za == OK)

###########################################################################################################################################
def ring_check(nums):
    #find the ratio disc(k)/disc(p) 
    p=poly(nums)
    f_p = p.make(p.coeff)
    ring_disc= f_p.discriminant()
    K= NumberField(f_p,'a')
    field_disc= K.discriminant()
    ratio= field_disc/ring_disc
    return(ratio)


#############################################################################################################################################

def ring_info(nums):
    #takes an array of 5 numbers and returns some information about the ring defined by the polynomial with this array as its coefficients
    p= poly(nums)
    f_p=p.make(p.coeff)
    print(f_p.discriminant())
    K = NumberField(f_p, 'a')
    print(K.discriminant())
    ########################
    print('___________________________________________________')
    find_monogens(nums)

    ###############################
    print('__________________________________________________')
    print('gal group is')
    print(gal_group(nums))
    ################################3
    print('__________________________________')
    print('ring int test')
    print(ring_int_test(nums))
    print('done')

Input: An array $[1,a_1, a_2, a_3, a_4]$ of integers.
        Soon, I will add the ability to have inputs of degree 3.

Constraints:The polynomial $f(t) =  t^4 + a_1 t^3 + a_2 t^2 + a_3 t + a_4$ must be irreducible.

Output: A dictionary that pairs the solutions to the cubic resolvent thue equations with a set of integer-valued triples $(x,y,z)$, such that $c + xt + yt^2 + zt^3$ is a monogenizer of the ring $\mathbb{Z}[t]/(f(t)). Every monogenization will be represented by exactly one triple in the output.


In [ ]:
P=poly([1,1,1,1,1])

In [3]:
find_monogens([1, -7, -6, -8, -6])

test
(1, 0)
(5, -1)
[1, 0]
{(1, 0): {(1, 0, 0)}, (5, -1): set()}


In [2]:
P=poly([1, 0, -4, -1, 1])
P.monogen_test()
print( P.solu_coords)


(-2, -1)
(1, 0)
(2, -1)
(4, -1)
[4, 4, 4, 5]
{(-2, -1): {(-12, 1, 3), (-5, 0, 1), (4, 9, -5), (8, 1, -2)}, (1, 0): {(1, 0, 0), (0, 2, 1), (-3, -1, 1), (-4, 0, 1)}, (2, -1): {(-3, 0, 1), (-8, 1, 2), (2, 1, -1), (4, 1, -1)}, (4, -1): {(0, 1, 0), (-1, -2, 1), (-14, -3, 4), (4, 33, 16), (-4, 1, 1)}}


In [3]:
print(P.make(P.cubic_res))

t^3 + 4*t^2 - 4*t - 17
